In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
anime_data = pd.read_csv("./../../data/fixed/anime_full.csv")

In [6]:
characters = pd.read_csv('./../../data/fixed/character_full.csv')

In [9]:
anime_data = anime_data.drop('synopsis', axis = 1)

# 장르'만' 가지고 유사도 측정

In [26]:
genre_set = set()

In [27]:
for idx in anime_data.index:
    genres = anime_data.loc[idx, 'Genres'].split(',')
    for genre in genres:
        genre_set.add(genre)

In [28]:
genre_list = list(genre_set)

In [29]:
score = np.zeros((anime_data['MAL_ID'].nunique(), len(genre_set)))

In [30]:
for idx in anime_data.index:
    temp = anime_data.loc[idx, 'Genres'].split(',')
    for i in range(len(genre_list)):
        if genre_list[i] in temp:
            score[idx,i] = 1.0

In [46]:
cosine_similar = cosine_similarity(score, score)
for i in range(len(cosine_similar)):
    cosine_similar[i,i] = 1.0

In [47]:
cosine_similar_df = pd.DataFrame(cosine_similar)

In [52]:
def ani_rec_genre(df, cosine_similar_df, ani_name, top_n = 20):
    ani_idx = anime_data[anime_data['Name'] == ani_name].index.values
    sorted_df = cosine_similar_df[ani_idx]
    sorted_df = sorted_df.sort_values(list(ani_idx), ascending = False)
    top_n_idx = sorted_df.index[:top_n+1]
    top_n_idx_without_input = list(set(top_n_idx) - set(ani_idx))
    print(anime_data.loc[top_n_idx_without_input][['Name', 'MAL_ID']])
    return list(anime_data.loc[top_n_idx_without_input]['MAL_ID'])

In [53]:
ani_rec_genre(anime_data, cosine_similar_df, 'Naruto')

NameError: name 'cosine_similar_df' is not defined

# 장르, Type, Source, Year 합산
year은 nan이 많아서 생략

In [17]:
anime_data.loc[10]

MAL_ID                                                   20
Name                                                 Naruto
Genres    Action,Adventure,Comedy,SuperPower,MartialArts...
Type                                                     TV
Source                                                Manga
Year                                                 2000.0
Name: 10, dtype: object

In [18]:
type_set = set()
for idx in anime_data.index:
    type_set.add(anime_data.loc[idx, 'Type'])
type_list = list(type_set)

source_set = set()
for idx in anime_data.index:
    source_set.add(anime_data.loc[idx, 'Source'])
source_list = list(source_set)

In [19]:
score_type = np.zeros((anime_data['MAL_ID'].nunique(), len(type_set)))
score_source = np.zeros((anime_data['MAL_ID'].nunique(), len(source_set)))

In [20]:
for idx in anime_data.index:
    temp = anime_data.at[idx,'Type']
    for i in range(len(type_list)):
        if type_list[i] == temp:
            score_type[idx,i] = 1.0

In [21]:
for idx in anime_data.index:
    temp = anime_data.at[idx,'Source']
    for i in range(len(source_list)):
        if source_list[i] == temp:
            score_source[idx,i] = 1.0

In [22]:
score_source.shape

(12181, 15)

In [31]:
score_full = np.hstack([score, score_type, score_source])

In [32]:
score_full.shape

(12181, 63)

In [33]:
cosine_similar_full = cosine_similarity(score_full, score_full)
for i in range(len(cosine_similar_full)):
    cosine_similar_full[i,i] = 1.0

In [34]:
cosine_similar_full_df = pd.DataFrame(cosine_similar_full)

In [50]:
result = ani_rec_genre(anime_data, cosine_similar_full_df, 'Naruto')

                                                    Name  MAL_ID
4352                                          Medaka Box   11761
131                                          Shaman King     154
8010                     Boruto: Naruto Next Generations   34566
3212                Dragon Ball Z: Atsumare! Gokuu World    6714
203                                       Rekka no Honoo     238
5518                              Dragon Ball Kai (2014)   22777
8268                     Boruto: Jump Festa 2016 Special   35072
1274                              Katekyo Hitman Reborn!    1604
1366                                  Naruto: Shippuuden    1735
1178                                          D.Gray-man    1482
3035                                     Dragon Ball Kai    6033
233                                               Bleach     269
684                                        Dragon Ball Z     813
428                                      Ueki no Housoku     479
1329                     

In [51]:
result

[11761,
 154,
 34566,
 6714,
 238,
 22777,
 35072,
 1604,
 1735,
 1482,
 6033,
 269,
 813,
 479,
 1685,
 42205,
 987,
 30694,
 904,
 223]